<a href="https://colab.research.google.com/github/rarodrigo-datasc/emenda-parlamentar-spark/blob/main/emenda_parlamentar_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar as dependências
!apt update -qq
!apt upgrade
!apt install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

46 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files bash binutils binutils-common binutils-x86-64-linux-gnu bsdutils coreutils
  cuda-compat-12-2 cuda-keyring cuda-toolkit-12-config-common cuda-toolkit-config-common dpkg
  dpkg-dev libbinutils libblkid1 libc-bin libctf-nobfd0 libctf0 libcups2 libdpkg-perl libgnutls30
  libldap-2.5-0 libmount1 libpam-modules libpam-modules-bin libpam-runtime libpam0g libperl5.34
  libprocps8 libsmartcols1 libudev1 linux-libc-dev login mount openssl passwd perl perl-base
  perl-modules-5.34 procps tar util-linux
42 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Need to get 59.9 MB of archives.
After this operation, 124 kB of additional disk space will be use

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init('spark-3.1.2-bin-hadoop2.7')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName("Spark Airnbnb").config('spark.ui.port', '4050').getOrCreate()
# https://insideairbnb.com/get-the-data/ dados do inside airbnb
!wget --quiet --show-progress http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/visualisations/listings.csv

listings.csv        100%[===================>]   6.30M  11.4MB/s    in 0.6s    


In [ ]:
df_spark = spark.read.csv("./listings.csv", inferSchema=True, header=True)

In [ ]:
df_spark

DataFrame[id: bigint, name: string, host_id: int, host_name: string, neighbourhood_group: string, neighbourhood: string, latitude: double, longitude: double, room_type: string, price: int, minimum_nights: int, number_of_reviews: int, last_review: string, reviews_per_month: double, calculated_host_listings_count: int, availability_365: int, number_of_reviews_ltm: int, license: string]